In [5]:
import numpy as np

W = np.random.randn(5, 3)  # example matrix
U, S, Vt = np.linalg.svd(W, full_matrices=False)

u = U[:, 0]      # Left singular vector
v = Vt[0, :]     # Right singular vector (note: Vt = V.T)
print(u)
print(v)
print(S[0])  # Singular value

[-0.4367726  -0.03793902 -0.31824461  0.6984112  -0.46768845]
[ 0.04641084 -0.2999522   0.9528246 ]
3.3651089804275913


In [6]:
def top_singular_vectors(W, num_iter=100):
    v = np.random.randn(W.shape[1])
    v /= np.linalg.norm(v)
    
    for _ in range(num_iter):
        u = W @ v
        u /= np.linalg.norm(u)
        v = W.T @ u
        v /= np.linalg.norm(v)
    
    return u, v

u, v = top_singular_vectors(W)
print(u)
print(v)
print(u.T @ W @ v)


[ 0.4367726   0.03793902  0.31824461 -0.6984112   0.46768845]
[-0.04641084  0.2999522  -0.9528246 ]
3.3651089804275918


In [64]:
import numpy as np
def u1s1v1t(W, num_iter=30):
    """Power iteration using PyTorch operations"""
    v = np.random.randn(W.shape[1])
    v /= np.linalg.norm(v)
    
    for _ in range(num_iter):
        u = W @ v
        u /= np.linalg.norm(u)
        v = W.T @ u
        v /= np.linalg.norm(v)
    sigma1 = (u.T @ W @ v)
    u = u.reshape(-1, 1)         # shape: (5, 1)
    v = v.reshape(-1, 1)         # shape: (3, 1)
    return sigma1 * u @ v.T

In [65]:
u1s1v1t(np.array([[1, 2, 3], [4, 5, 6]]), num_iter=30)

array([[1.57454629, 2.08011388, 2.58568148],
       [3.75936076, 4.96644562, 6.17353048]])